# Twitter API Social Listening

## Team : indigoSquad Proyecto: NATA

### Integrantes :
        * Azucena Sánchez
        * Arturo Ventura
        * Aaron Téllez
        * Evelyn Hernández        
        * Guillermo Villafuerte                
        * Guillermo Escalona

In [35]:
import tweepy
import pandas as pd
import googletrans
import matplotlib.pyplot as plt
from textblob import TextBlob
from googletrans import Translator
%matplotlib inline

In [16]:
ACCESS_TOKEN = '????????????'
ACCESS_SECRET = '????????????'
CONSUMER_KEY = '????????????'
CONSUMER_SECRET = '????????????'

def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)
    return api

api = connect_to_twitter_OAuth()

In [17]:
diccionario = ["cuenta", "tarjeta", "tc", "tdc", "retiro", "cajeros", "ejecutivo", "sucursal",
               "retuvo", "trabo", "servicio", "practicaja", "llamada", "llamaron", "marcado", "cargo",
               "puntos", "condusefmx", "condusef", "pyme", "pandemia", "credito", "cliente",
               "endeudado", "spam", "comision", "colgaron", "transferencia", "spei", "anualidad", "descuento",
               "atasco", "comunicarme", "token", "seguro", "asesor", "aclaracion", "poliza", "biometrico",
               "app", "aplicacion", "wallet", "audiomatico", "banco", "deposito", "transa", "netkey", "dolar",
               "divisa", "sat", "isr", "impuesto", "interes", "servicio", "respuesta", "atencion", "hipoteca",
               "casa", "vivienda", "pay", "pago", "pesimo", "bueno", "excelente", "afore", "nip", "crisis",
               "seguridad", "fraude", "acoso", "nomina", "reclama", "reclamo", "cambia", "cambio", "disculpa",
               "debito", "oferta", "ofrece", "queja", "lavado", "dinero", "efectivo", "robo", "roba", "extravio",
               "reembolso", "rembolso"]
    
def normalizar(texto):
    return texto.lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

def palabras_texto(texto):
    completo = texto.strip()
    coincide = []
    
    for palabra in diccionario:
        if completo.find(palabra) != -1:
            coincide.append(palabra)
            
    cadena = ""
    
    for palabra in coincide:
        cadena = cadena + palabra + ","
        
    return cadena[0:len(cadena) - 1]

In [18]:
def valores_tweet(status, banco):
    ret = {}

    ret["fecha"] = status.created_at
    ret["rt"] = status.retweet_count
    ret["fav"] = status.favorite_count
    ret["src"] = status.source
    ret["sigue_a"] = status.user.friends_count
    ret["lo_siguen"] = status.user.followers_count
    ret["desde"] = status.user.created_at
    ret["banco"] = banco
    
    if status.place is not None:
        ret["country"] = status.place.country_code or ""
        ret["type"] = status.place.place_type or ""
        ret["place"] = status.place.full_name or ""
    
    if (status.full_text.startswith("RT")):
        if hasattr(status, 'retweeted_status'):
            ret["texto"] = status.retweeted_status.full_text
        else:
            ret["texto"] = status.full_text
            
    else:
        ret["texto"] = status.full_text
        
    ret["texto"] = ret["texto"].replace('@', '').replace('#', '').replace('\n', ' ')
    ret["palabras"] = palabras_texto(ret["texto"])
    
    return ret

In [25]:
def todos_tweets(query, banco):
    data = []
    
    for tweet in tweepy.Cursor(api.search, q = query, lang = "es", tweet_mode = "extended").items(400):
        data.append(valores_tweet(tweet, banco))

    return data

In [27]:
try:
    archivoTuits = "tuits_07.csv"
    dataset = []

    for data in todos_tweets("bbva", "bbva"):
        dataset.append(data)
        
    for data in todos_tweets("citibanamex", "citibanamex"):
        dataset.append(data)
        
    for data in todos_tweets("banorte", "banorte"):
        dataset.append(data)
        
    for data in todos_tweets("hsbc", "hsbc"):
        dataset.append(data)
        
    for data in todos_tweets("scotiabank", "scotiabank"):
        dataset.append(data)
        
except TweepError:
    # Tragar excepción
    print("Error")
    
finally:
    df = pd.DataFrame(dataset, columns = ['banco', 'fecha', 'texto', 'src', 'rt', 'fav',
                                          'palabras', 'country', 'type', 'place',
                                          'desde', 'sigue_a', 'lo_siguen'])

    df.drop_duplicates('texto', inplace = True)

    with open(archivoTuits, "w", newline = "", encoding = "utf-8") as csv_file:
        df.to_csv(csv_file)
        csv_file.close()

In [ ]:
try:
    archivoUsuarios = "cuentas.csv"
    dataset = []
    users = ["bbva_mex", "citibanamex", "banorte_mx"]

    for handle in users:
        user = api.get_user(handle)
        cell = {}
        cell["nombre"] = user.name
        cell["usuario"] = user.screen_name
        cell["foto"] = user.profile_image_url_https
        cell["desc"] = user.description
        cell["desde"] = user.created_at
        cell["seguidores"] = user.followers_count
        cell["palomita"] = user.verified
        cell["tuits"] = user.statuses_count
        dataset.append(cell)

        
except TweepError:
    print("Error")
    
finally:
        
    df = pd.DataFrame(dataset, columns = ['nombre', 'usuario', 'foto', 'desc', 'desde', 'seguidores', 'palomita', 'tuits'])

    with open(archivoUsuarios, "w", newline = "", encoding = "utf-8") as csv_file:
        df.to_csv(csv_file)
        csv_file.close()

In [29]:
data = pd.read_csv("tuits_06.csv", sep=',')

In [30]:
data

,Unnamed: 0,banco,fecha,texto,src,rt,fav,palabras,country,type,place,desde,sigue_a,lo_siguen
0,0,bbva,2020-10-18 05:40:33,BBVA da a sus empleados el control de su forma...,Twitter for Android,0,0,token,NaN,NaN,NaN,2009-12-02 15:10:28,270,270
1,1,bbva,2020-10-18 05:38:23,morbobson El de enmedio está guapísimo trabaja...,Twitter for Android,0,0,NaN,NaN,NaN,NaN,2014-01-25 01:41:42,735,490
2,2,bbva,2020-10-18 05:37:00,cuando van a quitar todos los ambulantes de Ma...,Twitter for iPhone,1,0,"cliente,banco",NaN,NaN,NaN,2012-11-07 17:00:23,439,465
3,3,bbva,2020-10-18 05:35:01,"El BBVA, a partir de enero, va a cobrar 2 euro...",Twitter for Android,467,0,"comision,dinero",NaN,NaN,NaN,2013-11-19 17:23:53,906,316
4,4,bbva,2020-10-18 05:34:25,Sr lopezobrador_ el pueblo no está contento l...,Twitter for iPhone,0,0,"tarjeta,fraude,dinero,roba",NaN,NaN,NaN,2010-01-30 05:45:34,238,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,1995,scotiabank,2020-10-17 15:06:14,japocalde ¿Qué inconvenientes presentas con ap...,Sprinklr Publishing,0,0,"sucursal,servicio,app,servicio",NaN,NaN,NaN,2016-08-19 15:01:29,376,5767
1151,1996,scotiabank,2020-10-17 15:03:52,Con llamada falsa de Scotiabank le roban 557 m...,Twitter Web App,0,0,"tarjeta,llamada,pandemia,roba",NaN,NaN,NaN,2016-08-11 04:05:52,61,799
1152,1997,scotiabank,2020-10-17 15:02:47,Genial! Me alisto toda bonis para ir al scotia...,Twitter for iPhone,0,1,tarjeta,NaN,NaN,NaN,2011-01-18 18:46:29,424,238
1153,1998,scotiabank,2020-10-17 14:59:13,"sexxxydoll__ ScotiabankApoya Por otra parte, u...",Twitter for iPhone,0,0,NaN,NaN,NaN,NaN,2019-05-13 03:22:59,152,7


In [31]:
data.texto = data.texto.replace({'_':''}, regex=True)

In [36]:
translator = Translator()

In [41]:
def translateWord(word):
    return translator.translate(word, src='es', dest='en')

In [47]:
englishTextTweets = []
for word in data.texto:
    englishTextTweets.append(translateWord(word).text) 

In [48]:
englishTextTweets

['BBVA gives its employees control of their training with a system based on ‘tokens’ https://t.co/QQKowZfvx1 blockchain elearning',
 'morbobson The one in the middle is gorgeous, he works at BBVA I love seeing him',
 "When they are going to remove all of Matias Romero's street vendors between the university and Pythagoras, it is disgusting, they give the bank a terrible appearance, and there are criminals hiding, it is a danger for Bbva customers and to enter and leave the MetroCDMX STaboadaMx Tahipat vecletranvalle https: // t .co / PJPku89wQl",
 "As of January, BBVA will charge 2 euros for withdrawing money at the window and will raise commissions if the direct debit payroll is less than 800 euros. It hurts the profiles of precarious workers and pensioners. It's a fucking shame 😡😡😡",
 'Mr lopezobrador the people are not happy their money is being stolen through fraud on their credit cards SantanderMx ScotiabankMX BBVAMex the most affected hopefully there is still some instance to att

# Sentiment Analysis

In [65]:
# pip install nltk
import nltk
nltk.download('vader_lexicon')

import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aron\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [66]:
analyzer = SentimentIntensityAnalyzer()

In [67]:
analyzer.polarity_scores('Hello')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [77]:
scores = np.zeros(len(data.texto))

for phrase in englishTextTweets:
    # Measure the polarity of the tweet
    print(analyzer.polarity_scores(phrase))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.613, 'pos': 0.387, 'compound': 0.8481}
{'neg': 0.274, 'neu': 0.726, 'pos': 0.0, 'compound': -0.9432}
{'neg': 0.146, 'neu': 0.854, 'pos': 0.0, 'compound': -0.7574}
{'neg': 0.295, 'neu': 0.591, 'pos': 0.114, 'compound': -0.8365}
{'neg': 0.0, 'neu': 0.278, 'pos': 0.722, 'compound': 0.3818}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.107, 'neu': 0.893, 'pos': 0.0, 'compound': -0.4767}
{'neg': 0.063, 'neu': 0.887, 'pos': 0.05, 'compound': -0.1779}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.038, 'neu': 0.914, 'pos': 0.048, 'compound': 0.1531}
{'neg': 0.0, 'neu': 0.851, 'pos': 0.149, 'compound': 0.4215}
{'neg': 0.126, 'neu': 0.782, 'pos': 0.092, 'compound': -0.25}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{

{'neg': 0.192, 'neu': 0.808, 'pos': 0.0, 'compound': -0.8402}
{'neg': 0.0, 'neu': 0.812, 'pos': 0.188, 'compound': 0.4995}
{'neg': 0.05, 'neu': 0.95, 'pos': 0.0, 'compound': -0.3485}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.892, 'pos': 0.108, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.4215}
{'neg': 0.0, 'neu': 0.785, 'pos': 0.215, 'compound': 0.8225}
{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'compound': 0.8481}
{'neg': 0.307, 'neu': 0.693, 'pos': 0.0, 'compound': -0.8997}
{'neg': 0.053, 'neu': 0.901, 'pos': 0.046, 'compound': -0.0772}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.065, 'neu': 0.935, 'pos': 0.0, 'compound': -0.1511}
{'neg': 0.403, 'neu': 0.597, 'pos': 0.0, 'compound': -0.9749}
{'neg': 0.39, 'neu': 0.61, 'pos': 0.0, 'compound': -0.9749}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.068, 'neu': 0.932, 'pos': 0.0, 'compound': -0.2732}
{'neg': 0.148, 'neu': 0.852, 'pos': 0.0